In [16]:
# Import all necessary libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import seaborn as sns
%matplotlib inline
from shapely.geometry import Polygon
from shapely.geometry import Point,Polygon
from pyproj import Proj, Transformer
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from geopy.geocoders import GoogleV3
import fiona
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import json
import requests
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
from shapely.ops import nearest_points
from pyproj import CRS
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
from functools import lru_cache
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
from sklearn.neighbors import BallTree
import numpy as np


#!pip install pyarrow
#!pip install geopy
#!pip install fiona

### Galveston

In [17]:
# Function to check if a DataFrame is a GeoDataFrame and has valid geometries
def check_geodataframe(gdf, name):
    # Check if it is a GeoDataFrame
    if isinstance(gdf, gpd.GeoDataFrame):
        print(f"'{name}' is a GeoDataFrame.")
    else:
        print(f"'{name}' is not a GeoDataFrame.")
        return  # Stop further checks if it's not a GeoDataFrame
    
    # Check if the geometry column exists and has valid geometries
    if 'geometry' in gdf.columns:
        print(f"'{name}' contains a 'geometry' column.")
        # Check for any invalid geometries
        if gdf['geometry'].is_valid.all():
            print(f"All geometries in '{name}' are valid.")
        else:
            print(f"Some geometries in '{name}' are invalid.")
    else:
        print(f"'{name}' does not contain a 'geometry' column.")

In [18]:
def calculate_units(row):
    """
    Calculate the estimated number of units in a building based on its square footage and property type.

    Parameters:
    - row (pd.Series): A pandas Series representing a row of a DataFrame, expected to contain
                       the building's property type and its total square footage.

    Returns:
    - float or None: The estimated number of units in the building if the property type is recognized
                     and square footage is available. Returns None if the property type is not recognized
                     or square footage is missing.

    The function uses predefined average unit areas for different property types to estimate the number of units.
    If the property type is not in the predefined list or the square footage is not provided, the function returns None.
    """
    average_unit_areas = {
        'Single Family': 2299,
        'Multifamily': 1046,
        'Condo': 592,
        'Apartment': 592,
        'Townhouse': 592,
        'Manufactured': 2000,
        'Land': 2000
    }
    building_type = row.get('propertyType')
    average_area = average_unit_areas.get(building_type, None)
    if average_area is not None and 'SQFEET' in row and row['SQFEET'] is not None:
        return row['SQFEET'] / average_area
    else:
        return None

In [19]:
# Load API Data
api_data = pd.read_json('/Users/kd6801/Desktop/Texas_RentCast_data/TX_Apar.json')

In [20]:
len(api_data)

1206

In [26]:
#api_data.iloc[1]

id                  2101-Church-Rear-St,-Galveston,-TX-77550
formattedAddress    2101 Church Rear St, Galveston, TX 77550
addressLine1                             2101 Church Rear St
addressLine2                                            None
city                                               Galveston
state                                                     TX
zipCode                                                77550
county                                      Galveston County
latitude                                            29.30409
longitude                                          -94.79115
propertyType                                       Apartment
yearBuilt                                             1927.0
lastSaleDate                                             NaN
squareFootage                                            NaN
lotSize                                                  NaN
assessorID                                               NaN
legalDescription        

In [13]:
#api_data.iloc[2]

In [29]:
# Load FEMA data
# Path to my File Geodatabase
gdb_path = '/Users/kd6801/Desktop/Texas-Fema/TX_Structures.gdb'

# List all layers in the File Geodatabase
layers = fiona.listlayers(gdb_path)


# Read a specific layer into a GeoDataFrame
gdf = gpd.read_file(gdb_path, layer=layers[0])

# The FEMA Data is too large to load, we need to 

IndexError: list index out of range

In [6]:
# Load building foot-traffic data
path_to_Galveston_parquet = '/Users/kd6801/Desktop/foot-traffic/galveston_full_patterns.parquet'
foot =  pd.read_parquet(path_to_Galveston_parquet, engine='pyarrow')


In [ ]:
# Data Pre-processing, joining FEMA with API 
api_data['geometry'] = api_data.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
gdf_api = gpd.GeoDataFrame(api_data, geometry='geometry')
gdf_api.set_crs('EPSG:4326',inplace=True)
joined = gpd.sjoin(gdf_api, gdf, how="inner", op='intersects')
# Extract 'architectureType' from the 'features' dictionaries into a new column
joined = joined.dropna(subset=['HEIGHT','SQFEET','features'])
joined['architectureType'] = joined['features'].apply(lambda x: x.get('architectureType', None))


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the building footprints
joined.plot(ax=ax, color='lightgrey', edgecolor='black', alpha=0.5, label='Building Footprints')

# Plot the locations of large apartments
large_apartments_gdf.plot(ax=ax, color='cyan', markersize=10, label='Large Apartments')

# Create custom patches
building_patch = mpatches.Patch(color='lightgrey', label='Building Footprints')
apartment_patch = mpatches.Patch(color='cyan', label='Large Apartments')

# Add the custom patches to the legend
ax.legend(handles=[building_patch, apartment_patch])

# Add axis labels
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Add a title
ax.set_title('Large Residential Buildings Within Building Footprints for Hilo, HI (Using 3000 Threshold)')

# Optional: remove the axis for a cleaner look
ax.set_axis_off()

# Show the plot
plt.show()
